In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [2]:
# Numpy のアップグレード
!pip install numpy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


ランタイムを再起動してください

In [1]:
# 環境変数を設定
!echo 'export LD_LIBRARY_PATH=/usr/lib' >> ~/.bashrc
!source ~/.bashrc

In [2]:
import pandas_datareader.data as pdr

def get_stock_data(code):
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [3]:
import plotly.graph_objs as go
import datetime as dt
import pandas as pd
import talib as ta

name = '積水ハウス'
code = 1928
df = get_stock_data(code)
close = df['Close']

ma5, ma25 = ta.SMA(close, timeperiod=5), ta.SMA(close, timeperiod=25)
df['ma5'], df['ma25'] = ma5, ma25

rdf = df[dt.datetime(2021,12,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime('%m-%d-%Y')

layout = {
            "title"  : { "text": "{} {}".format(code, name), "x":0.5 }, 
            "xaxis" : { "title": "日付", "rangeslider": { "visible": False },  "type" : "category" },
            "yaxis" : { "title": "価格（円）", "side": "left", "tickformat": "," },
            "plot_bgcolor":"light blue"
          }

data =  [
            # ローソク足
            go.Candlestick(x=rdf.index, open=rdf["Open"], high=rdf["High"], low=rdf["Low"], close=rdf["Close"],   
                            increasing_line_color="red", decreasing_line_color="gray"),
            # 5日移動平均線
            go.Scatter(x=rdf.index, y=rdf["ma5"], name="MA5", line={ "color": "royalblue", "width": 1.2 } ),
            # 25日移動平均線
            go.Scatter(x=rdf.index, y=rdf["ma25"], name="MA25", line={ "color": "lightseagreen", "width": 1.2 } )
        ]

fig = go.Figure(data = data, layout = go.Layout(layout))
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()